<img width="8%" alt="HubSpot.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/HubSpot.png" style="border-radius: 15%">

# HubSpot - Create task from Chat
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Create+task+from+Chat:+Error+short+description">Bug report</a>

**Tags:** #hubspot #sales #crm #engagements #task #command

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-11-22 (Created: 2023-11-22)

**Description:** This notebook demonstrates how to create a task in HubSpot from Naas Chat.

**References:**
- [HubSpot API - Tasks](https://developers.hubspot.com/docs/api/crm/tasks)
- [Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)

## Input

### Import libraries

In [ ]:
import requests
import naas
from datetime import datetime, date, timezone, timedelta

### Setup variables
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `task_subject`: The title of the task.
- `task_type`: The type of task. Values include EMAIL, CALL, or TODO.
- `task_body`: The task notes.
- `task_priority`: The priority of the task. Values include LOW, MEDIUM, or HIGH.
- `task_status`: The status of the task, either NOT_STARTED, COMPLETED, IN_PROGRESS, WAITING, DEFERRED
- `owner_email`: Email of the owner name of the task.
- `due_date`: Due date in format YYYY-MM-DD or integer between current date. By default, the hour set will be 9:00 AM.
- `body`: This variable stores the body to be send by the webhook.

In [ ]:
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
task_subject = "Connect"
task_type = "TODO"
task_body = ""
task_priority = "MEDIUM"
task_status = "NOT_STARTED"
owner_email = None
due_date = None
body = {}

### Setup parameters
The webhook body will be injected below this cell when the webhook is triggered. 
Therefore, it is important to set up how you will handle the injected variable from the body in order to make your script work.
To receive the body from the webhook, please ensure that this cell is tagged as "parameters".

In [ ]:
# Parameters
if len(body) > 0:
    task_subject = body.get("task_subject")
    task_body = body.get("task_body")
    task_type = body.get("task_type")
    task_priority = body.get("task_priority")
    owner_email = body.get("owner_email")
    due_date = body.get("due_date")

## Model

### Get HubSpot owner ID from owner name

In [ ]:
def get_owners(hs_access_token):
    # Request
    headers = {
        'accept': "application/json",
        'content-type': "application/json",
        'authorization': f"Bearer {hs_access_token}"
    }
    url = "https://api.hubapi.com/crm/v3/owners"

    # Response
    res = requests.get(url, headers=headers)
    return res.json()['results']

hubspot_owner_id = None
for owner in get_owners(hs_access_token):
    email = owner.get("email")
    if email == owner_email:
        hubspot_owner_id = owner.get("id")
        break
        
print("Owner ID:", hubspot_owner_id)

### Transform due date to timestamp

In [ ]:
hs_timestamp = None
due_date = 1
try:
    date_object = datetime.strptime(due_date, "%Y-%m-%d")
    print("The string matches the ISO format for a date.")
except:
    date_object = datetime.now() + timedelta(days=1)
    print("The string does not match the ISO format for a date.")
    
hs_timestamp = date_object.replace(tzinfo=timezone.utc, hour=9, minute=0, second=0, microsecond=0).strftime("%s") + "000"
print("Timestamp:", hs_timestamp)

### Create task

In [ ]:
def create_task(
    hs_access_token,
    hs_task_subject,
    hs_task_type="TODO",
    hs_task_body=None,
    hs_task_priority=None,
    hs_task_status=None,
    hs_timestamp=None,
    hubspot_owner_id=None,
    contact_ids=[],
    company_ids=[],
    deal_ids=[]
):
    # Init
    status = "ok"
    message = ""
    data = []
    associations = []
    
    try:    
        # Get the current timestamp in UTC
        if not hs_timestamp:
            hs_timestamp = datetime.utcnow().replace(tzinfo=timezone.utc).strftime("%s") + "000"

        # Create contact associations
        contacts = []
        for object_id in contact_ids:
            contacts.append(
                {
                    "to": {"id": object_id},
                    "types": [
                        {
                            "associationCategory": "HUBSPOT_DEFINED",
                            "associationTypeId": 204
                        }
                    ]
                }
            )

        # Create contact asso
        companies = []
        for object_id in company_ids:
            companies.append(
                {
                    "to": {"id": object_id},
                    "types": [
                        {
                            "associationCategory": "HUBSPOT_DEFINED",
                            "associationTypeId": 192
                        }
                    ]
                }
            )

        # Create contact asso
        deals = []
        for object_id in deal_ids:
            deals.append(
                {
                    "to": {"id": object_id},
                    "types": [
                        {
                            "associationCategory": "HUBSPOT_DEFINED",
                            "associationTypeId": 216
                        }
                    ]
                }
            )

        # Requests
        associations = contacts + companies + deals
        payload = {
            "properties":
            {
                "hs_task_subject": hs_task_subject,
                "hs_task_type": hs_task_type,
                "hs_task_body": hs_task_body,
                "hs_task_status": hs_task_status,
                "hs_task_priority": hs_task_priority,
                "hs_timestamp": hs_timestamp,
                "hubspot_owner_id": hubspot_owner_id,
            },
            "associations": associations
        }
        headers = {
            'accept': "application/json",
            'content-type': "application/json",
            'authorization': f"Bearer {hs_access_token}"
        }
        url = "https://api.hubapi.com/crm/v3/objects/tasks"

        # Response
        res = requests.post(url, headers=headers, json=payload)
        if res.status_code == 201:
            data = res.json()
            message = f"✅ Task successfully created: {data.get('id')}"
        else:
            print(res)
            message = res.text
        print(message)
    except Exception as e:
            message = f"Template error: {e}"
    return status, message

status, message = create_task(
    hs_access_token,
    task_subject,
    hs_task_type=task_type,
    hs_task_body=task_body,
    hs_task_priority=task_priority,
    hs_task_status=task_status,
    hs_timestamp=hs_timestamp,
    hubspot_owner_id=hubspot_owner_id,
)

## Output

### Create or update Webhook

In [ ]:
naas.webhook.add()

### Return JSON response
Response sent to the browser before displayed in Chat UI.

In [ ]:
naas.webhook.respond_json(
    {
        "status": status, 
        "message": message
    }
)